# Assignment 3

Please answer each question to the corresponding question cell below. Your final code must have the code as well as the output of your code. You can use Saint Peter's [Azure Databricks](https://adb-7130196131129306.6.azuredatabricks.net/?o=7130196131129306#) to do this assignment.

## Questions

### Q1

What type of deployment configurations available in Spark? Evaluate each different mode in terms of interactivity, driver specs, cluster manager, and production use.

hen for execution, we submit a spark job to local or on a cluster, the behaviour of spark job totally depends on one parameter, that is the “Driver” component. Where “Driver” component of spark job will reside, it defines the behaviour of spark job.

Basically, there are two types of “Deploy modes” in spark, such as “Client mode” and “Cluster mode”. Let’s discuss each in detail.

**1. Spark Client Mode**

As we discussed earlier, the behaviour of spark job depends on the “driver” component. So here,”driver” component of spark job will run on the machine from which job is submitted. Hence, this spark mode is basically “client mode”.

When job submitting machine is within or near to “spark infrastructure”. Since there is no high network latency of data movement for final result generation between “spark infrastructure” and “driver”, then, this mode works very fine.

When job submitting machine is very remote to “spark infrastructure”, also have high network latency. Hence, in that case, this spark mode does not work in a good manner.

**2. Spark Cluster Mode**

Similarly, here “driver” component of spark job will not run on the local machine from which job is submitted. Hence, this spark mode is basically “cluster mode”. In addition, here spark job will launch “driver” component inside the cluster.

When job submitting machine is remote from “spark infrastructure”. Since, within “spark infrastructure”, “driver” component will be running. Thus, it reduces data movement between job submitting machine and “spark infrastructure”. In such case, This mode works totally fine.

While we work with this spark mode, the chance of network disconnection between “driver” and “spark infrastructure”  reduces. Since they reside in the same infrastructure. Also, reduces the chance of job failure.

**3. Running Spark Applications on YARN**

YARN controls resource management, scheduling, and security when we run spark applications on it. It is possible to run an application in any mode here, whether it is cluster mode or client mode.

In addition, while we run spark on YARN, spark executor runs as a YARN container. There is a case where MapReduce schedules a container and starts a JVM for each task. There spark hosts multiple tasks within the same container. Hence, it enables several orders of magnitude faster task startup time.

**Spark Deploy Modes in YARN**

Each application instance has an ApplicationMaster process, in YARN. That is generally the first container started for that application. However, the application is responsible for requesting resources from the ResourceManager.

As soon as resources are allocated, the application instructs NodeManagers to start containers on its behalf. For an active client, ApplicationMasters eliminate the need.

Basically, the process starting the application can terminate. Also, the coordination continues from a process managed by YARN running on the cluster.

**1. Cluster Deployment Mode**

When the driver runs in the applicationmaster on a cluster host, which YARN chooses, that spark mode is a cluster mode. It signifies that process, which runs in a YARN container, is responsible for various steps.

Such as driving the application and requesting resources from YARN. Hence, the client that launches the application need not continue running for the complete lifespan of the application.

*Note: For using spark interactively, cluster mode is not appropriate. Since applications which require user input need the spark driver to run inside the client process, for example, spark-shell and pyspark. That initiates the spark application.*

**2. Client Deployment Mode**

When the driver runs on the host where the job is submitted, that spark mode is a client mode. To request executor containers from YARN, the ApplicationMaster is merely present here. To schedule works the client communicates with those containers after they start.

Reference: https://sparkbyexamples.com/spark/spark-deploy-modes-client-vs-cluster/

In Short:

| Mode            | Spark Driver                                     | Spark Executor                                          |CLuster Manager |
|-----------------|-----------------                                 |-----------------                                        |-----------------|
|Local            |Runs on a single JVM, like a laptop or single node|Runs on the same JVM as the driver                       |Runs on the same host|
|Standalone       |Can run on any node in the cluster                |Each node in the cluster will launch its own executor JVM|Can be allocated arbitrarily to any host in the cluster|
|YARN (client)    |Runs on a client, not part of the cluster         |YARN’s NodeManager’s container                           |YARN’s Resource Manager works with YARN’s Application Master to allocate the containers on NodeManagers for executors|
|YARN (cluster)   |Runs with the YARN Application Master             |Same as YARN client mode                                 |Same as YARN client mode|
|Kubernetes       |Runs in a Kubernetes pod                          |Each worker runs within its own pod                      |Kubernetes Master|

### Q2

What is a driver program in Spark, what are the responsibilities of this program, and how many driver processes can exist in a Spark cluster?

A driver program is responsible for orchestrating parallel operations on the Spark cluster. The driver accesses the distributed components in the cluster—the Spark executors and cluster manager—through a SparkSession.As the part of the Spark application responsible for instantiating a SparkSession, the Spark driver has multiple roles: it communicates with the cluster manager; it requests resources (CPU, memory, etc.) from the cluster manager for Spark’s executors (JVMs); and it transforms all the Spark operations into DAG computations, schedules them, and distributes their execution as tasks across the Spark executors. Once the resources are allocated, it communicates directly with the executors.

In Spark architecture, there will be only one driver for the spark application.

### Q3

What is an executor, task and slots? Define each of them, write down it's responsibilities.

**Executor:**

A Spark executor runs on each worker node in the cluster. The executors communicate with the driver program and are responsible for executing tasks on the workers. In most deployments modes, only a single executor runs per node. The executors are responsible for actually carrying out the work that the driver assigns them. This means that each executor is responsible for only two things: executing code assigned to it by the driver, and reporting the state of the computation on that executor back to the driver node.

**Task:**

A task is a unit of work that will be sent to one executor. Each task is a combination of blocks of data and a set of transformations that will run on a single executor. Each stage is comprised of Spark tasks (a unit of execution), which are then federated across each Spark executor; each task maps to a single core and works on a single partition of data. As such, an executor with 16 cores can have 16 or more tasks working on 16 or more partitions in parallel, making the execution of Spark’s
tasks exceedingly parallel!

**Slots:**

Each executor nodes has slots which are logical execution cores for running tasks in parallel. The driver sends tasks to the vacant slots on the executors when there is work to be executed. This ties to distributing and parallelizing an execution thread.

### Q4

What is a Spark DataFrame, how it is different than a table in a SQL database?

A DataFrame is the most common Structured API and simply represents a table of data with rows and columns. The list that defines the columns and the types within those columns is called the schema.

In Spark, a DataFrame is a distributed collection of data organized into named columns. It is conceptually equivalent to a table in a relational SQL database or a data frame in R/Python, but with richer optimizations under the hood. DataFrames can be constructed from a wide array of sources such as: structured data files, tables in Hive, external databases, or existing RDDs.

### Q5

What is the difference between wide and narrow transformations, what is the role of partitions in this difference? Give example functions to each type of transformation.

**Narrow transformation —** Any transformation where a single output partition can be computed from a single input partition is a narrow transformation. In Narrow transformation, all the elements that are required to compute the records in single partition live in the single partition of parent RDD. A limited subset of partition is used to calculate the result. Narrow transformations are the result of map(), filter().
In this case, data need not be shuffled and the partitions remain the same. filter() and contains() perform narrow transformations.

**Wide transformation —** Any transformation where a single output partition can be computed from more than one input partition is a wide transformation. In wide transformation, all the elements that are required to compute the records in the single partition may live in many partitions of parent RDD. The partition may live in many partitions of parent RDD. Wide transformations are the result of groupbyKey and reducebyKey.
In this case, the transformation will force a shuffle of data from each of the executor’s partitions across the cluster. groupBy() or orderBy() perform wide transformations.

### Q6

What is the cost model in The Catalyst Optimizer?

The Catalyst optimizer takes a computational query and converts it into an execution
plan. It goes through four transformational phases:
1. Analysis
2. Logical optimization
3. Physical planning
4. Code generation
**Phase 1: Analysis**

The Spark SQL engine begins by generating an abstract syntax tree (AST) for the SQL or DataFrame query. In this initial phase, any columns or table names will be resolved by consulting an internal Catalog, a programmatic interface to Spark SQL that holds a list of names of columns, data types, functions, tables, databases, etc. Once they’ve all been successfully resolved, the query proceeds to the next phase.

**Phase 2: Logical optimization**

This phase comprises two internal stages. Applying a standardrule based optimization approach, the __Catalyst optimizer__ will first construct a set of multiple plans and then, using its cost-based optimizer (CBO), assign costs to each plan.

*A cost-based optimizer is about using costs of query plans in order to determine which query plan to run with.*

These plans are laid out as operator trees; they may include, for example, the process of constant folding, predicate pushdown, projection pruning, Boolean expression simplification, etc. This logical plan is the input into the physical plan.

**Phase 3: Physical planning**

In this phase, Spark SQL generates an optimal physical plan for the selected logical plan, using physical operators that match those available in the Spark execution engine. In the physical planning phase, Catalyst may generate multiple plans and compare them based on cost. All other phases are purely rule-based. Each phase uses different types of tree nodes; Catalyst includes libraries of nodes for expressions, data types, and logical and physical operators.

_In the physical planning phase, Spark SQL takes a logical plan and generates one or more physical plans, using physical operators that match the Spark execution engine. It then selects a plan using a cost model. At the moment, cost-based optimization is only used to select join algorithms: for relations that are known to be small, Spark SQL uses a broadcast join, using a peer-to-peer broadcast facility available in Spark. The framework supports broader use of cost-based optimization, however, as costs can be estimated recursively for a whole tree using a rule. We thus intend to implement richer cost-based optimization in the future._

The physical planner also performs rule-based physical optimizations, such as pipelining projections or filters into one Spark map operation. In addition, it can push operations from the logical plan into data sources that support predicate or projection pushdown. We will describe the API for these data sources in a later section.


**Phase 4: Code generation**

The final phase of query optimization involves generating efficient Java bytecode to run on each machine. Because Spark SQL can operate on data sets loaded in memory, Spark can use state-of-the-art compiler technology for code generation to speed up execution. In other words, it acts as a compiler. Project Tungsten, which facilitates whole-stage code generation, plays a role here.

### Q7

Explain what happens when a user submits a Spark Application that initiates a SparkSesion and calls an action. Elaborate the role of driver, Jobs, Stages, and Tasks.

Once I have a SparkSession, I can program Spark using the APIs to perform Spark operations or actions. If I launch the Spark shell locally on my laptop, all the operations/ actions would run locally, in a single JVM. But I can just as easily launch a Spark shell to analyze data in parallel on a cluster as in local mode. 

**Spark Driver**

At the core of every Spark application is the Spark driver program, which creates a SparkSession object. When I am working with a Spark shell, the driver is part of the shell and the SparkSession object (accessible via the variable spark) is created for me.

**Spark Jobs**

During interactive sessions with Spark shells, the driver converts my Spark application into one or more Spark jobs. It then transforms each job into a DAG. This, in essence, is Spark’s execution plan, where each node within a DAG could be a single or multiple Spark stages.

**Spark Stages**

As part of the DAG nodes, stages are created based on what operations/ actions can be performed serially or in parallel. Not all Spark operations/ actions can happen in a single stage, so they may be divided into multiple stages. Often stages are delineated on the operator’s computation boundaries, where they dictate data transfer among Spark executors.

**Spark Tasks**

Each stage is comprised of Spark tasks (a unit of execution), which are then federated across each Spark executor; each task maps to a single core and works on a single partition of data. As such, an executor with 16 cores can have 16 or more tasks working on 16 or more partitions in parallel, making the execution of Spark’s tasks exceedingly parallel!

Answer the following questions using the dataset below.

In [0]:
df = spark \
    .read \
    .format("csv")\
    .option("header", "true") \
    .load("/databricks-datasets/airlines/part-00000")

### Q8

Show the number of columns, and the number of rows in a tuple, ex: Pandas has `df.shape`.

In [0]:
(len(df.columns), df.count())

Out[2]: (31, 645918)

In [0]:
# Alternative
df.toPandas().shape

Out[3]: (645918, 31)

### Q9

Show all the unique `UniqueCarrier`'s in this data.

In [0]:
df.select("UniqueCarrier").distinct().show()

+-------------+
UniqueCarrier|
+-------------+
 UA|
 EA|
 PS|
 NW|
 HP|
 TW|
 WN|
 PI|
 AA|
 DL|
 CO|
 PA (1)|
 US|
 AS|
+-------------+

### Q10

Show the total number of delays for each `UniqueCarrier`.

In [0]:
df.groupBy("UniqueCarrier").count().show()

+-------------+------+
UniqueCarrier| count|
+-------------+------+
 UA|101654|
 EA| 50994|
 PS| 27945|
 NW| 71932|
 HP| 29965|
 TW| 45948|
 WN| 41975|
 PI| 63080|
 AA| 56091|
 DL| 63104|
 CO| 42756|
 PA (1)| 10749|
 US| 32293|
 AS| 7432|
+-------------+------+

### Q11 - Bonus

Show the total number of delays for each `UniqueCarrier` for all the files shown below.

In [0]:
dbutils.fs.ls('/databricks-datasets/airlines/')

Out[6]: [FileInfo(path='dbfs:/databricks-datasets/airlines/README.md', name='README.md', size=1089, modificationTime=1454697889000),
 FileInfo(path='dbfs:/databricks-datasets/airlines/_SUCCESS', name='_SUCCESS', size=0, modificationTime=1436493184000),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00000', name='part-00000', size=67108879, modificationTime=1436493184000),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00001', name='part-00001', size=67108862, modificationTime=1436493185000),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00002', name='part-00002', size=67108930, modificationTime=1436493185000),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00003', name='part-00003', size=67108804, modificationTime=1436493186000),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00004', name='part-00004', size=67108908, modificationTime=1436493186000),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00005', name='part-00005', size=67108890, modificationTime=1436493187000),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00006', name='part-00006', size=67108825, modificationTime=1436493187000),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00007', name='part-00007', size=67108880, modificationTime=1436493187000),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00008', name='part-00008', size=67108832, modificationTime=1436493188000),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00009', name='part-00009', size=67108857, modificationTime=1436493188000),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00010', name='part-00010', size=67108931, modificationTime=1436493189000),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00011', name='part-00011', size=67108872, modificationTime=1436493189000),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00012', name='part-00012', size=67108791, modificationTime=1436493189000),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00013', name='part-00013', size=67108856, modificationTime=1436493190000),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00014', name='part-00014', size=67108855, modificationTime=1436493190000),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00015', name='part-00015', size=67108875, modificationTime=1436493191000),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00016', name='part-00016', size=67108898, modificationTime=1436493191000),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00017', name='part-00017', size=67108873, modificationTime=1436493191000),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00018', name='part-00018', size=67108881, modificationTime=1436493192000),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00019', name='part-00019', size=67108829, modificationTime=1436493192000),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00020', name='part-00020', size=67108863, modificationTime=1436493193000),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00021', name='part-00021', size=67108868, modificationTime=1436493193000),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00022', name='part-00022', size=67108827, modificationTime=1436493193000),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00023', name='part-00023', size=67108926, modificationTime=1436493194000),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00024', name='part-00024', size=67108885, modificationTime=1436493194000),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00025', name='part-00025', size=67108843, modificationTime=1436493195000),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00026', name='part-00026', size=67108837, modificationTime=1436493195000),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00027', name='part-00027', size=67108911, modificationTime=1436493195000),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00028', name='part-0002

In [0]:
from pyspark.sql.types import *
 
# Create schema for csv imports
main_schema = [i for i in df.schema]
main_columns = StructType(main_schema)

In [0]:
df_big = spark.read.format("csv").option("header", True).schema(main_columns).load("/databricks-datasets/airlines/part-*")

In [0]:
df_big.groupBy("UniqueCarrier").count().show()

+-------------+---------+
UniqueCarrier| count|
+-------------+---------+
 UA|132997956|
 EA| 9197835|
 PI| 8739560|
 PS| 836169|
 AA|149846230|
 NW|102926108|
 EV| 16971693|
 B6| 8113396|
 HP| 36366757|
 TW| 37577415|
 DL|165478422|
 OO| 30908487|
 F9| 3369574|
 YV| 8540549|
 TZ| 2084197|
 US|140755085|
 AQ| 1543808|
 MQ| 39548888|
 OH| 14641741|
 HA| 2742648|
+-------------+---------+
only showing top 20 rows

## Notes

After done with the assignment, please upload your work to the blackboard. Please make sure you upload:

- The outputs of the code in the notebook
- The pdf version of the notebook

Best of luck!